## Programación Lineal
### Plan de Rebalanceo de Carteras

En este cuaderno se muestra la resolución de un problema que plantea determinar que traspasos de dinero
entre fondos de inversión hay que realizar para llevar una cartera actual a una cartera objetivo.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp

### Datos
Tenemos disponible un CSV de ejemplo que contiene el valor de las posiciones
de la cartera actual y la objetivo, que nos servirán de ejemplo. Además
tenemos otro CSV con los días que tardaría un traspaso, y que nos servirá
como ejemplo de coste

In [ ]:
portfolio = pd.read_csv('../data/portfolio_to_rebalance.csv', index_col='fondo')
portfolio

In [ ]:
portfolio.sum()

In [ ]:
dias_traspaso = pd.read_csv('../data/coste_traspaso.csv', index_col=0)
dias_traspaso

In [ ]:
def rebalancing_series(source, target):
    """
    Calcula los flujos de entrada y salida para rebalancear
    la cartera a partir de la diferencia entre la actual
    y el objetivo
    """
    diff = target - source
    outflow = -diff[diff < 0]
    inflow = diff[diff > 0]
    
    return outflow, inflow

In [ ]:
outfunds, infunds = rebalancing_series(portfolio.current, portfolio.target)

In [ ]:
outfunds

In [ ]:
infunds

Seleccionamos de los datos de coste los necesarios para construir
el problema

In [ ]:
costes_traspaso = dias_traspaso.loc[outfunds.index, infunds.index]
costes_traspaso

### Implementación del modelo de optimización


En este caso hemos hecho coincidir el valor de la carteras.  Las
restricciones con desigualdad nos permitirían encontrar soluciones
en los casos por ejemplo que tenemos holgura en la cartera actual

In [ ]:
out_n = outfunds.shape[0]
in_n = infunds.shape[0]

In [ ]:
fundmove = cp.Variable((out_n, in_n))

In [ ]:
objetivo = cp.sum(cp.multiply(fundmove, costes_traspaso.values))

In [ ]:
constraints = []

for out_i, ofund in enumerate(outfunds):
    o_const = cp.sum(fundmove[out_i, :]) <= ofund
    constraints.append(o_const)
for in_j, infund in enumerate(infunds):
    in_const = cp.sum(fundmove[:, in_j]) >= infund
    constraints.append(in_const)

In [ ]:
constraints.append(fundmove >= 0)

In [ ]:
problem = cp.Problem(cp.Minimize(objetivo), constraints)

In [ ]:
out_prob = problem.solve()

Reconstruimos la solución a partir de los valores que se quedan 
almacenados en las variables de decisión

In [ ]:
resultado = pd.DataFrame(fundmove.value.round(2),
                         index=outfunds.index,
                         columns=infunds.index)    
resultado

___

### Propuesta Ejercicios
1. Modificar el modelo para que de forma puntual ningún traspaso
sea mayor de 9000€.
